<a href="https://colab.research.google.com/github/sp7412/colab/blob/master/distilling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def load_mnist():
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.
  x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.

  y_train = to_categorical(y_train.astype('float32'))
  y_test = to_categorical(y_test.astype('float32'))

  return (x_train, y_train), (x_test, y_test)

In [ ]:
def build_simple_model():
  inputs = layers.Input(shape=(28, 28, 1))
  x = layers.Conv2D(32, kernel_size=(3,3), activation='relu', name='Conv1')(inputs)
  x = layers.Conv2D(64, kernel_size=(3,3), activation='relu', name='Conv2')(x)
  
  x = layers.MaxPooling2D(pool_size=(2,2), name='MaxPool')(x)
  x = layers.Dropout(rate=0.25, name='Dropout1')(x)
  
  x = layers.Flatten(name='Flat')(x)
  x = layers.Dense(128, activation='relu', name='FC1')(x)
  x = layers.Dropout(rate=0.5, name='Dropout2')(x)
  
  x = layers.Dense(10, name='logits')(x)
  preds = layers.Activation('softmax', name='Softmax')(x)
  
  model = Model(inputs=inputs, outputs=preds)
  model.summary()
  return model

In [ ]:
model = build_simple_model()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 26, 26, 32)        320       
_________________________________________________________________
Conv2 (Conv2D)               (None, 24, 24, 64)        18496     
_________________________________________________________________
MaxPool (MaxPooling2D)       (None, 12, 12, 64)        0         
_________________________________________________________________
Dropout1 (Dropout)           (None, 12, 12, 64)        0         
_________________________________________________________________
Flat (Flatten)               (None, 9216)              0         
_________________________________________________________________
FC1 (Dense)                  (None, 128)              

In [ ]:
(x_train, y_train), (x_test, y_test) = load_mnist()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=15, batch_size=512)

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/15
118/118 [==============================] - 3s 22ms/step - loss: 0.3612 - accuracy: 0.8881
Epoch 2/15
118/118 [==============================] - 3s 22ms/step - loss: 0.1047 - accuracy: 0.9685
Epoch 3/15
118/118 [==============================] - 3s 22ms/step - loss: 0.0724 - accuracy: 0.9786
Epoch 4/15
118/118 [==============================] - 3s 22ms/step - loss: 0.0579 - accuracy: 0.9829
Epoch 5/15
118/118 [==============================] - 3s 22ms/step - loss: 0.0489 - accuracy: 0.9852
Epoch 6/15
118/118 [==============================] - 3s 22ms/step - loss: 0.0432 - accuracy: 0.9872
Epoch 7/15
118/118 [==============================] - 3s 22ms/step - loss: 0.0384 - accuracy: 0.9881
Epoch 8/15
118/118 [==============================] - 3s 22ms/step - loss: 0.0332 - accuracy: 0.9895
Epoch 9/15
118/118 [==============================] - 3s 22ms/step - loss: 0.0320 - accuracy: 0.9901
Epoch 10/15
118/118 [=====

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.0291 - accuracy: 0.9920
Test Loss: 0.029050899669528008
Test Accuracy: 0.9919999837875366


In [ ]:
def softmax_with_temperature(logits, temperature=1):
  logits = logits / temperature
  return np.exp(logits) / np.sum(np.exp(logits))

In [ ]:
model_sans_softmax = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = model_sans_softmax.predict(x_train)

In [ ]:
unsoftened_train_prob = softmax_with_temperature(model_logits, 1)

In [ ]:
def build_small_model():
  inputs = layers.Input(shape=(28, 28, 1))
  x = layers.Flatten()(inputs)
  x = layers.Dense(128, activation='relu', name='FC1')(x)
  x = layers.Dense(128, activation='relu', name='FC2')(x)
  x = layers.Dense(10, name='logits')(x)
  preds = layers.Activation('softmax', name='Softmax')(x)

  model = Model(inputs=inputs, outputs=preds)
  model.summary()
  return model

In [ ]:
small_model = build_small_model()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
FC1 (Dense)                  (None, 128)               100480    
_________________________________________________________________
FC2 (Dense)                  (None, 128)               16512     
_________________________________________________________________
logits (Dense)               (None, 10)                1290      
_________________________________________________________________
Softmax (Activation)         (None, 10)                0         
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
________________________________________________

In [ ]:
small_model.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
small_model.fit(x_train, y_train, epochs=50, batch_size=512)

test_loss, test_acc = small_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Epoch 1/50
118/118 [==============================] - 0s 3ms/step - loss: 0.4736 - accuracy: 0.8683
Epoch 2/50
118/118 [==============================] - 0s 3ms/step - loss: 0.2005 - accuracy: 0.9411
Epoch 3/50
118/118 [==============================] - 0s 3ms/step - loss: 0.1449 - accuracy: 0.9574
Epoch 4/50
118/118 [==============================] - 0s 3ms/step - loss: 0.1105 - accuracy: 0.9675
Epoch 5/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0901 - accuracy: 0.9735
Epoch 6/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0739 - accuracy: 0.9781
Epoch 7/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0616 - accuracy: 0.9809
Epoch 8/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0532 - accuracy: 0.9835
Epoch 9/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0445 - accuracy: 0.9865
Epoch 10/50
118/118 [==============================] - 0s 3ms/step - loss: 0.0393 - accuracy: 0.9886

In [ ]:
model_sans_softmax = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = model_sans_softmax.predict(x_train)
temperature = 4.0
softened_train_prob = softmax_with_temperature(model_logits, temperature)

In [ ]:
new_small_model = build_small_model()

Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
FC1 (Dense)                  (None, 128)               100480    
_________________________________________________________________
FC2 (Dense)                  (None, 128)               16512     
_________________________________________________________________
logits (Dense)               (None, 10)                1290      
_________________________________________________________________
Softmax (Activation)         (None, 10)                0         
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_______________________________________________

In [ ]:
logits = new_small_model.get_layer('logits').output
logits = layers.Lambda(lambda x: x / temperature, name='Temperature')(logits)
preds = layers.Activation('softmax', name='Softmax')(logits)
  
new_small_model = Model(inputs=new_small_model.input, outputs=preds)
new_small_model.summary()

Model: "functional_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
FC1 (Dense)                  (None, 128)               100480    
_________________________________________________________________
FC2 (Dense)                  (None, 128)               16512     
_________________________________________________________________
logits (Dense)               (None, 10)                1290      
_________________________________________________________________
Temperature (Lambda)         (None, 10)                0         
_________________________________________________________________
Softmax (Activation)         (None, 10)              

In [ ]:
new_small_model.compile(optimizer=RMSprop(lr=0.03), loss='categorical_crossentropy', metrics=['accuracy'])
new_small_model.fit(x_train, softened_train_prob, epochs=100, batch_size=512)

test_loss, test_acc = new_small_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Epoch 1/100
118/118 [==============================] - 0s 3ms/step - loss: 0.5913 - accuracy: 0.0984
Epoch 2/100
118/118 [==============================] - 0s 3ms/step - loss: 3.2234 - accuracy: 0.0995
Epoch 3/100
118/118 [==============================] - 0s 3ms/step - loss: 7.3755 - accuracy: 0.1021
Epoch 4/100
118/118 [==============================] - 0s 3ms/step - loss: 13.3597 - accuracy: 0.0995
Epoch 5/100
118/118 [==============================] - 0s 3ms/step - loss: 21.2360 - accuracy: 0.1022
Epoch 6/100
118/118 [==============================] - 0s 3ms/step - loss: 30.9339 - accuracy: 0.1001
Epoch 7/100
118/118 [==============================] - 0s 3ms/step - loss: 42.2653 - accuracy: 0.0975
Epoch 8/100
118/118 [==============================] - 0s 3ms/step - loss: 54.9498 - accuracy: 0.0995
Epoch 9/100
118/118 [==============================] - 0s 3ms/step - loss: 68.7000 - accuracy: 0.0998
Epoch 10/100
118/118 [==============================] - 0s 3ms/step - loss: 85.5446 -